In [67]:
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

In [68]:
# DataFrame des vols
flight_data = {
    'departure_city': ['Paris', 'Paris', 'Nice', 'Nice'],
    'destination_city': ['Nice', 'Nice', 'Paris', 'Paris'],
    'start_date': ['2024-06-01', '2024-06-01', '2024-06-01', '2024-06-01'],
    'duration': [2, 1.5, 1.75, 2],
    'stopovers': [0, 1, 0, 0],
    'price': [100, 150, 120, 130],
}
flight_df = pd.DataFrame(flight_data)

# DataFrame des hôtels
hotel_data = {
    'city': ['Nice', 'Lyon', 'Marseille'],
    'check_in_date': ['2024-06-01', '2024-06-01', '2024-06-01'],
    'price_per_night': [60, 50, 55],
    'rating': [4.5, 4.0, 4.2]
}
hotel_df = pd.DataFrame(hotel_data)

# DataFrame des villes avec les thèmes
city_data = {
    'city': ['Nice', 'Lyon', 'Marseille'],
    'beach': [True, False, True],
    'family': [True, True, True],
    'ski': [False, False, False],
    'golf': [True, False, True]
}
city_df = pd.DataFrame(city_data)


In [69]:
# Modèle pour les vols
# Pré-traitement des données
flight_df['start_date'] = pd.to_datetime(flight_df['start_date'])
flight_df['start_day'] = flight_df['start_date'].dt.day
flight_df['start_month'] = flight_df['start_date'].dt.month
flight_df['start_year'] = flight_df['start_date'].dt.year

flight_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['duration', 'start_day', 'start_month', 'start_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['departure_city', 'destination_city', 'stopovers'])
    ])

# Pipeline de modélisation pour les vols
flight_model = Pipeline(steps=[
    ('preprocessor', flight_preprocessor),
    ('regressor', RandomForestRegressor())
])

# Split des données
X_flight = flight_df.drop(columns=['price', 'start_date'])
y_flight = flight_df['price']
X_flight_train, X_flight_test, y_flight_train, y_flight_test = train_test_split(X_flight, y_flight, test_size=0.2, random_state=42)

# Entraînement du modèle
flight_model.fit(X_flight_train, y_flight_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['duration', 'start_day',
                                                   'start_month',
                                                   'start_year']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['departure_city',
                                                   'destination_city',
                                                   'stopovers'])])),
                ('regressor', RandomForestRegressor())])

In [70]:
# Modèle pour les hôtels
# Pré-traitement des données
hotel_df['check_in_date'] = pd.to_datetime(hotel_df['check_in_date'])
hotel_df['check_in_day'] = hotel_df['check_in_date'].dt.day
hotel_df['check_in_month'] = hotel_df['check_in_date'].dt.month
hotel_df['check_in_year'] = hotel_df['check_in_date'].dt.year

hotel_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['check_in_day', 'check_in_month', 'check_in_year']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['city'])
    ])

# Pipeline de modélisation pour les hôtels
hotel_model = Pipeline(steps=[
    ('preprocessor', hotel_preprocessor),
    ('regressor', RandomForestRegressor())
])

# Split des données
X_hotel = hotel_df.drop(columns=['price_per_night', 'check_in_date', 'rating'])
y_hotel = hotel_df['price_per_night']
X_hotel_train, X_hotel_test, y_hotel_train, y_hotel_test = train_test_split(X_hotel, y_hotel, test_size=0.2, random_state=42)

# Entraînement du modèle
hotel_model.fit(X_hotel_train, y_hotel_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['check_in_day',
                                                   'check_in_month',
                                                   'check_in_year']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city'])])),
                ('regressor', RandomForestRegressor())])

In [71]:
def predict_trip_cost(departure_city, destination_city, start_date, end_date, budget, themes, flight_model, hotel_model, city_df):
    # Préparer les données pour la prédiction des vols
    start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

    flight_outbound = {
        'departure_city': [departure_city],
        'destination_city': [destination_city],
        'duration': [2],  # Exemple de durée de vol, vous pouvez ajuster selon les données réelles
        'stopovers': [0],  # Exemple sans escale
        'start_day': [start_date_dt.day],
        'start_month': [start_date_dt.month],
        'start_year': [start_date_dt.year]
    }
    flight_inbound = {
        'departure_city': [destination_city],
        'destination_city': [departure_city],
        'duration': [2],  # Exemple de durée de vol, vous pouvez ajuster selon les données réelles
        'stopovers': [0],  # Exemple sans escale
        'start_day': [end_date_dt.day],
        'start_month': [end_date_dt.month],
        'start_year': [end_date_dt.year]
    }

    flight_outbound_df = pd.DataFrame(flight_outbound)
    flight_inbound_df = pd.DataFrame(flight_inbound)

    outbound_price = flight_model.predict(flight_outbound_df)[0]
    inbound_price = flight_model.predict(flight_inbound_df)[0]

    # Préparer les données pour la prédiction des hôtels
    duration_days = (end_date_dt - start_date_dt).days

    hotel = {
        'city': [destination_city],
        'check_in_day': [start_date_dt.day],
        'check_in_month': [start_date_dt.month],
        'check_in_year': [start_date_dt.year]
    }

    hotel_df = pd.DataFrame(hotel)

    hotel_price_per_night = hotel_model.predict(hotel_df)[0]
    total_hotel_price = hotel_price_per_night * duration_days

    # Calculer le coût total du séjour
    total_cost = outbound_price + inbound_price + total_hotel_price

    # Vérifier les thèmes
    city_info = city_df[city_df['city'] == destination_city].iloc[0]
    city_themes = {
        'beach': city_info['beach'],
        'family': city_info['family'],
        'ski': city_info['ski'],
        'golf': city_info['golf']
    }

    for theme in themes:
        if not city_themes.get(theme, False):
            return {'error': f'The destination does not support the theme: {theme}'}

    if total_cost > budget:
        return {'error': 'The trip exceeds the budget'}

    return {
        'outbound_flight_price': outbound_price,
        'inbound_flight_price': inbound_price,
        'total_hotel_price': total_hotel_price,
        'total_cost': total_cost
    }

# Exemple d'utilisation
departure_city = 'Paris'
destination_city = 'Nice'
start_date = '2024-06-01'
end_date = '2024-06-10'
budget = 1000
themes = ['beach', 'family']

trip_cost = predict_trip_cost(departure_city, destination_city, start_date, end_date, budget, themes, flight_model, hotel_model, city_df)
print(trip_cost)


{'outbound_flight_price': 107.6, 'inbound_flight_price': 125.5, 'total_hotel_price': 470.25, 'total_cost': 703.35}
